In [1]:
!git clone https://github.com/dbachko/bhagavad-gita-as-it-is.git

Cloning into 'bhagavad-gita-as-it-is'...
remote: Enumerating objects: 2232, done.
remote: Total 2232 (delta 0), reused 0 (delta 0), pack-reused 2232
Receiving objects: 100% (2232/2232), 1.98 MiB | 5.48 MiB/s, done.
Resolving deltas: 100% (777/777), done.


In [120]:
import glob 
collection = glob.glob("./bhagavad-gita-as-it-is/en/*/*.md",recursive=True)

In [121]:
cleared = filter(lambda x: "README" not in x, collection)

In [122]:
import pandas as pd

df = pd.DataFrame(columns = ["section","verse","translation","purport","embedding"])

i = 0
for x in cleared:
    i+=1
    with open(x) as ofile:
        data = ofile.read()
        section = '-'.join(x.split("/")[-2:])
        p1 = data[:data.find("\n\n\n\n### Translation:\n\n**")][len("# Text 2\n\n"):]
        p2 = data[data.find("### Translation:\n\n**"):data.find("**\n\n### Purport:")][len("### Translation:\n\n**"):]
        p3 = data[data.find("### Purport:"):][len("### Purport:\n\n"):]
        df.loc[i] = [section,p1,p2,p3,""]

In [123]:
df.tail()

,section,verse,translation,purport,embedding
657,14-8.md,तमस्त्वज्ञानजं विद्धि मोहनं सर्वदेहिनाम् । \n...,"O son of Bharata, know that the mode of darkne...",In this verse the specific application of the ...,
658,14-3.md,मम योनिर्महद्ब्रह्म तस्मिन्गर्भं दधाम्यहम् । ...,"The total material substance, called Brahman, ...",This is an explanation of the world: everythin...,
659,14-7.md,रजो रागात्मकं विद्धि तृष्णासङ्गसमुद्भवम् । \n...,The mode of passion is born of unlimited desir...,The mode of passion is characterized by the at...,
660,14-6.md,तत्र सत्त्वं निर्मलत्वात्प्रकाशकमनामयम् । \nस...,"O sinless one, the mode of goodness, being pur...",The living entities conditioned by material na...,
661,14-2.md,इदं ज्ञानमुपाश्रित्य मम साधर्म्यमागताः । \nसर...,"By becoming fixed in this knowledge, one can a...",After acquiring perfect transcendental knowled...,


In [124]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
openai.api_key=""#Add Your Key here
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [125]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    return {
        idx: get_embedding(r.purport) for idx, r in df.iterrows()
    }

In [146]:
def get_top_document(query,matrix):
    dot_products = np.dot(matrix, query)
    max_index = np.argsort(dot_products)[-2:]
    return max_index

In [127]:
document_embeddings_final = compute_doc_embeddings(df)

In [129]:
for x in document_embeddings_final.keys():
    df.loc[x]['embedding'] = document_embeddings_final[x]

In [130]:
df.to_csv("final_gita_embedded.csv")

In [132]:
matrix = np.array(df['embedding'].tolist())
document_index = get_top_document(matrix[3],matrix)

In [155]:
query_embedding = get_embedding("How to achieve success in life?")

In [156]:
document_index = get_top_document(query_embedding,matrix)

In [157]:
df.iloc[document_index[1]]['purport']

'In the pursuit of self-realization, one has to follow many rules and regulations to control the mind and the senses and to concentrate the mind on the self. All these procedures are very difficult, bitter like poison, but if one is successful in following the regulations and comes to the transcendental position, he begins to drink real nectar, and he enjoys life.'

In [158]:
df.iloc[document_index[0]]['purport']

'One should be very careful of these three enemies to human life: lust, anger and greed. The more a person is freed from lust, anger and greed, the more his existence becomes pure. Then he can follow the rules and regulations enjoined in the Vedic literature. By following the regulative principles of human life, one gradually raises himself to the platform of spiritual realization. If one is so fortunate, by such practice, to rise to the platform of Krishna consciousness, then success is guaranteed for him. In the Vedic literature, the ways of action and reaction are prescribed to enable one to come to the stage of purification. The whole method is based on giving up lust, greed and anger. By cultivating knowledge of this process, one can be elevated to the highest position of self-realization; this self-realization is perfected in devotional service. In that devotional service, the liberation of the conditioned soul is guaranteed. Therefore, according to the Vedic system, there are in